<a href="https://colab.research.google.com/github/rafa408/Projeto_Bootcamp_CDIA/blob/main/Tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Estratégia de Tratamento de Dados**



**Estratégia de Tratamento de Dados**

1. Valores Ausentes

Ação:
Para colunas numéricas (x_maximo, soma_da_luminosidade, maximo_da_luminosidade, espessura_da_chapa_de_aço, index_quadrado, indice_global_externo, indice_de_luminosidade):

* Avaliar a porcentagem de valores ausentes em cada coluna.
* Se a porcentagem for baixa (e.g., &lt; 5%), imputar com a mediana (robusta a outliers) ou média (se a distribuição for aproximadamente simétrica).
* Se a porcentagem for moderada (e.g., 5% - 20%), considerar a imputação com métodos mais sofisticados (e.g., KNN imputer, regressão) ou criar um indicador booleano para "valor ausente".
* Se a porcentagem for alta (e.g., > 20%), avaliar se a coluna é realmente útil para o modelo. Se não for, considerar removê-la.
* Para colunas categóricas (tipo_do_aço_A400):
Imputar com a moda (valor mais frequente) ou criar uma nova categoria "Desconhecido".
Justificativa:
* Valores ausentes podem introduzir viés e prejudicar o desempenho do modelo.
* A escolha do método de imputação depende da natureza dos dados e da quantidade de valores ausentes.

2. Valores Negativos

Ação:
Para colunas de luminosidade (soma_da_luminosidade, maximo_da_luminosidade, indice_de_luminosidade):
* Manter os valores negativos, pois são considerados coerentes.
Para as demais colunas numéricas:
* Investigar a distribuição dos valores negativos.
* Se forem poucos e dispersos, substituir por zero ou pela mediana/média dos valores positivos.
* Se houver um padrão (e.g., muitos valores negativos em um intervalo específico), investigar a causa e aplicar a correção apropriada (e.g., somar um valor constante para torná-los positivos).

Justificativa:

Valores negativos em medidas físicas como área ou perímetro não fazem sentido e devem ser tratados como erros.
Valores negativos de luminosidade podem ter um significado físico (e.g., em relação a um ponto de referência) e, portanto, devem ser mantidos se forem coerentes com o problema.

3. Inconsistências em Colunas Categóricas

Ação:
* Identificar e unificar representações diferentes da mesma categoria (e.g., "Sim", "sim", "SIM", 1; "Não", "não", "NÃO", 0; "nao", "nao").
* Converter colunas booleanas para 0 e 1 (False para 0, True para 1).

Justificativa:

Inconsistências em categorias dificultam a análise e a modelagem.
Unificar representações garante que o modelo interprete corretamente as categorias.

4. Desbalanceamento de Classes (Se Aplicável)

Ação:
Se houver uma coluna de classe e ela estiver desbalanceada:
* Considerar técnicas de balanceamento durante a modelagem (e.g., oversampling da classe minoritária, undersampling da classe majoritária, pesos de classe).

Justificativa:

Modelos tendem a ter desempenho pior em classes minoritárias.
Balanceamento ajuda a melhorar a capacidade do modelo de generalizar para todas as classes.

5. Outliers

Ação:
* Identificar outliers usando métodos visuais (boxplots, scatter plots) e estatísticos (Z-score, IQR).
* Avaliar o impacto dos outliers na distribuição dos dados e nos modelos.
* Decidir como tratar os outliers:
* Remover (se forem erros claros).
* Transformar (e.g., log) para reduzir a influência.
* Imputar (se forem poucos e não extremos).
* Manter (se forem valores válidos e importantes).

Justificativa:

Outliers podem distorcer a análise estatística e prejudicar o desempenho do modelo.
O tratamento adequado depende da natureza dos outliers.

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from imblearn.over_sampling import SMOTE # Importando SMOTE


def tratar_dados(df):
    """
    Realiza o tratamento de dados no DataFrame, incluindo:
    - Padronização de colunas binárias
    - Tratamento de valores ausentes
    - Tratamento de valores negativos
    - Criação da coluna 'anomalia_sensor'
    - Tratamento de outliers
    - Criação da coluna 'tipo_do_aco'
    - Remoção de colunas desnecessárias
    - Balanceamento de classes (se aplicável)
    """

    # 0. Copia o DataFrame para não modificar o original diretamente
    df = df.copy()

    # 1. Padronizar nomes de colunas (opcional, caso haja variações)
    df.columns = df.columns.str.strip().str.lower()

    # Lista de colunas binárias que precisam de tratamento
    colunas_binarias = [
        'tipo_do_aço_a300', 'tipo_do_aço_a400',
        'falha_1', 'falha_2', 'falha_3', 'falha_4', 'falha_5', 'falha_6', 'falha_outros'
    ]

    # Função para padronizar valores binários
    def padronizar_binario(valor):
        if pd.isna(valor):
            return None  # Manter como None para tratamento posterior
        valor = str(valor).strip().lower()
        if valor in ['sim', 's', 'yes', 'y', '1', 'true', 'verdadeiro', 'v', 'true.', 'sim.', 'yes.', '1.0', 1, True]:
            return 1
        elif valor in ['não', 'nao', 'no', 'n', '0', 'false', 'falso', 'f', 'false.', 'não.', 'nao.', '0.0', 0, False]:
            return 0
        else:
            return None  # Manter como None para tratamento posterior

    # Aplicar a padronização em todas as colunas binárias
    for coluna in colunas_binarias:
        if coluna in df.columns:
            df[coluna] = df[coluna].apply(padronizar_binario)
        else:
            print(f"Aviso: Coluna {coluna} não encontrada no dataset.")

    # Tratamento especial para tipo_do_aço_A300 e tipo_do_aço_A400
    if 'tipo_do_aço_a300' in df.columns and 'tipo_do_aço_a400' in df.columns:
        # Verificar valores nulos
        mask_null_a300 = df['tipo_do_aço_a300'].isna()
        mask_null_a400 = df['tipo_do_aço_a400'].isna()

        # Caso 1: Quando A300 é nulo, usar o inverso de A400 (se disponível)
        df.loc[mask_null_a300 & ~mask_null_a400, 'tipo_do_aço_a300'] = (
            1 - df.loc[mask_null_a300 & ~mask_null_a400, 'tipo_do_aço_a400']
        )

        # Caso 2: Quando A400 é nulo, usar o inverso de A300 (se disponível)
        df.loc[mask_null_a400 & ~mask_null_a300, 'tipo_do_aço_a400'] = (
            1 - df.loc[mask_null_a400 & ~mask_null_a300, 'tipo_do_aço_a300']
        )

        # Caso 3: Quando ambos são nulos, definir como padrão (A300=0, A400=1)
        both_null = mask_null_a300 & mask_null_a400
        df.loc[both_null, 'tipo_do_aço_a300'] = 0
        df.loc[both_null, 'tipo_do_aço_a400'] = 1

        # Garantir que não há linhas com ambos = 1 ou ambos = 0
        conflict_mask = (df['tipo_do_aço_a300'] == df['tipo_do_aço_a400'])
        df.loc[conflict_mask & (df['tipo_do_aço_a300'] == 1), 'tipo_do_aço_a400'] = 0
        df.loc[conflict_mask & (df['tipo_do_aço_a300'] == 0), 'tipo_do_aço_a400'] = 1

    # Tratar valores nulos nas outras colunas binárias (definir como 0)
    other_binary_cols = [col for col in colunas_binarias if col not in ['tipo_do_aço_a300', 'tipo_do_aço_a400']]
    for col in other_binary_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    # 2. Valores Ausentes
    # Imputar com mediana para colunas numéricas específicas
    for col in ['x_maximo', 'espessura_da_chapa_de_aço', 'index_quadrado']:
        if col in df.columns:
            df[col] = df[col].fillna(df[col].median())  # Correção: Atribuição direta

    # Imputação KNN para outras colunas numéricas
    imputer_knn = KNNImputer(n_neighbors=5)
    for col in ['soma_da_luminosidade', 'maximo_da_luminosidade', 'indice_global_externo', 'indice_de_luminosidade']:
        if col in df.columns:
            df[col] = imputer_knn.fit_transform(df[[col]])

    # Imputar com a moda para colunas categóricas
    for col in ['tipo_do_aço_a400']:  # Não há mais nulos em 'tipo_do_aço_a400', mas deixo aqui por precaução
        if col in df.columns:
            df[col] = df[col].fillna(df[col].mode()[0])  # Correção: Atribuição direta

    # 3. Valores Negativos
    # Tratar valores negativos em colunas específicas (além das de luminosidade)
    colunas_anomalias = [
        "x_minimo", "x_maximo", "y_minimo", "y_maximo",
        "area_pixels", "perimetro_x", "perimetro_y",
        "comprimento_do_transportador", "espessura_da_chapa_de_aço"
    ]
    for col in colunas_anomalias:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: max(0, x))  # Substitui negativos por 0

    # 4. Criar a coluna 'anomalia_sensor'
    df["anomalia_sensor"] = df[colunas_anomalias].lt(0).any(axis=1).astype(int)

    # 5. Tratamento de Outliers (Z-Score)
    colunas_outliers = ['x_minimo', 'x_maximo', 'y_minimo', 'y_maximo', 'area_pixels', 'perimetro_x', 'perimetro_y',
                        'soma_da_luminosidade', 'maximo_da_luminosidade', 'espessura_da_chapa_de_aço',
                        'temperatura', 'index_de_bordas', 'index_vazio', 'index_quadrado', 'index_externo_x',
                        'indice_de_bordas_x', 'indice_de_bordas_y', 'indice_de_variacao_x', 'indice_de_variacao_y',
                        'indice_global_externo', 'log_das_areas', 'log_indice_x', 'log_indice_y', 'indice_de_orientaçao',
                        'indice_de_luminosidade', 'sigmoide_das_areas', 'minimo_da_luminosidade',
                        'comprimento_do_transportador']  # Todas as numéricas

    for col in colunas_outliers:
        if col in df.columns:
            z_scores = np.abs(stats.zscore(df[col]))
            df = df[(z_scores < 3)].copy()  # Mantém apenas os valores dentro de 3 desvios padrão # Correção: Adicionado .copy()
            print(f"Outliers na coluna {col} tratados (Z-score < 3).")

    # 6. Criar a coluna 'tipo_do_aço' baseada na relação entre as duas colunas binárias
    if 'tipo_do_aço_a300' in df.columns and 'tipo_do_aço_a400' in df.columns:
        df['tipo_do_aço'] = 'Nenhum'  # Valor padrão

        # Caso onde tipo_do_aço_a300 = 1
        df.loc[df['tipo_do_aço_a300'] == 1, 'tipo_do_aço'] = 'A300'

        # Caso onde tipo_do_aço_a400 = 1
        df.loc[df['tipo_do_aço_a400'] == 1, 'tipo_do_aço'] = 'A400'

        # Remover colunas não necessárias
        colunas_para_remover = ['peso_da_placa', 'tipo_do_aço_a300', 'tipo_do_aço_a400']
        for col in colunas_para_remover:
            if col in df.columns:
                df.drop(col, axis=1, inplace=True)

    # 7. Balanceamento de Classes (Se Aplicável) - Opcional, requer coluna 'class'
    if 'class' in df.columns:
        print("Verificando balanceamento de classes...")
        num_classes = df['class'].nunique()
        class_counts = df['class'].value_counts()
        print(f"Número de classes: {num_classes}")
        print("Contagem de amostras por classe:")
        print(class_counts)

        # Se houver desbalanceamento (exemplo: se uma classe tiver menos de 10% das amostras)
        min_class_count = class_counts.min()
        total_samples = len(df)
        if min_class_count / total_samples < 0.1:  # Exemplo de limiar: 10%
            print("Desbalanceamento detectado. Aplicando oversampling da classe minoritária...")
            smote = SMOTE(random_state=42)  # Mantive o random_state para reprodutibilidade
            X = df.drop('class', axis=1)
            y = df['class']
            X_resampled, y_resampled = smote.fit_resample(X, y)
            df_balanced = pd.concat([X_resampled, y_resampled], axis=1)
            print("Balanceamento concluído.")
            return df_balanced  # Retorna o dataframe balanceado
        else:
            print("Classes balanceadas ou desbalanceamento não significativo.")
            return df  # Retorna o dataframe original
    else:
        print("Coluna 'class' não encontrada. Pulando balanceamento de classes.")
        return df

    return df  # Retorna o dataframe


# Carregar os dados
try:
    df = pd.read_csv("bootcamp_train.csv")
except FileNotFoundError:
    print("Erro: Arquivo 'bootcamp_train.csv' não encontrado. Verifique se ele está no mesmo diretório ou forneça o caminho correto.")
    exit()

# Tratar os dados
df_tratado = tratar_dados(df)

# Salvar o dataset tratado
df_tratado.to_csv('bootcamp_train_tratado.csv', index=False)
print("\nDataset tratado salvo como 'bootcamp_train_tratado.csv'")


Outliers na coluna x_minimo tratados (Z-score < 3).
Outliers na coluna x_maximo tratados (Z-score < 3).
Outliers na coluna y_minimo tratados (Z-score < 3).
Outliers na coluna y_maximo tratados (Z-score < 3).
Outliers na coluna area_pixels tratados (Z-score < 3).
Outliers na coluna perimetro_x tratados (Z-score < 3).
Outliers na coluna perimetro_y tratados (Z-score < 3).
Outliers na coluna soma_da_luminosidade tratados (Z-score < 3).
Outliers na coluna maximo_da_luminosidade tratados (Z-score < 3).
Outliers na coluna espessura_da_chapa_de_aço tratados (Z-score < 3).
Outliers na coluna temperatura tratados (Z-score < 3).
Outliers na coluna index_de_bordas tratados (Z-score < 3).
Outliers na coluna index_vazio tratados (Z-score < 3).
Outliers na coluna index_quadrado tratados (Z-score < 3).
Outliers na coluna index_externo_x tratados (Z-score < 3).
Outliers na coluna indice_de_bordas_x tratados (Z-score < 3).
Outliers na coluna indice_de_bordas_y tratados (Z-score < 3).
Outliers na coluna